### Compare Listing 
<li>vector uniform</li>
<li>greedy</li>
<li>e - greedy</li>
<li>Linear Reward Penalty</li>
<li>Linear Reward Inaction</li>
<li>UBC</li>
<li>BayesianUCB</li>
<li>Thompson Sampling</li>
<li>Neural Network</li>
<li>Non Stationary</li>

In [6]:
# import lib
import numpy as np
import matplotlib.pyplot as plt
import scipy,time
np.random.seed(5678)
np.set_printoptions(3)

In [12]:
# setting the ground truth
gt_prob = np.random.uniform(0,1,0)
print(gt_prob)

[0.844 1.    0.956 0.932 0.211 0.367 0.743 0.635 0.598 0.261 0.301 0.499
 0.713 0.01  0.902 0.842 0.679 0.867 0.636 0.299]


# Reference 
1. numpy.set_printoptions — NumPy v1.14 Manual. (2019). Docs.scipy.org. Retrieved 13 January 2019, from https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.set_printoptions.html